In [2]:
import pandas as pd
import numpy as np
import setting
import gc

gc.enable()

DATA = '~/Data/Molecular'

data = pd.concat([
    pd.read_pickle(f"{DATA}/basic.gz"),
    pd.read_pickle(f"{DATA}/angle_feature.gz"),
    pd.read_pickle(f"{DATA}/criskiev_distance_feature.gz"),
    pd.read_pickle(f"{DATA}/qm9.gz")
], axis = 1)
data['atom_2'] = data['atom_2'].astype('category')
data['atom_3'] = data['atom_3'].astype('category')
data['atom_4'] = data['atom_4'].astype('category')
data['atom_5'] = data['atom_5'].astype('category')
data['atom_6'] = data['atom_6'].astype('category')
data['atom_7'] = data['atom_7'].astype('category')
data['atom_8'] = data['atom_8'].astype('category')
data['atom_9'] = data['atom_9'].astype('category')
data = data.iloc[:4658147, :]
data = data.drop(columns=[
    'id',
    'molecule_name',
    'atom_index_0',
    'atom_index_1',
    'type',
    'atom_0',
    'x_0',
    'y_0',
    'z_0',
    'atom_1',
    'x_1',
    'y_1',
    'z_1'
])

train = pd.read_csv(f"{DATA}/train.csv", dtype={
    'molecule_name': 'category',
    'atom_index_0': 'int8',
    'atom_index_1': 'int8',
    'type': 'category',
    'scalar_coupling_constant': 'float32'
}, usecols=['type', 'scalar_coupling_constant'])
y = train.scalar_coupling_constant

In [4]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

X, Xv, yt, yv = train_test_split(data[train['type']=='1JHC'], y[train['type']=='1JHC'], test_size=0.3, random_state=233)

In [5]:
X.head()

,dist_x,dist_y,dist_z,dist,dist_x_molecule_mean,dist_x_molecule_std,dist_x_molecule_min,dist_x_molecule_max,dist_x_molecule_median,dist_x_molecule_skew,...,gap,zpve,Cv,freqs_min,freqs_max,freqs_mean,mulliken_min,mulliken_max,mulliken_atom_0,mulliken_atom_1
3071317,0.718120,0.629735,0.538759,1.096597,1.131857,0.772748,0.006586,3.370190,0.942428,0.708842,...,0.3157,0.204864,35.928001,62.010601,3114.227295,1427.378784,-0.408300,0.294869,0.095476,-0.313909
788545,0.415129,0.415989,0.928545,1.098897,1.346787,0.878814,0.020909,3.357167,1.216717,0.558615,...,0.1682,0.138216,31.857000,118.599297,3664.471191,1348.222534,-0.580987,0.410183,0.126495,-0.512462
1920166,0.613223,0.408835,0.809255,1.094570,1.092615,0.746989,0.050205,3.087011,0.937903,0.722839,...,0.1983,0.169389,33.361000,57.035999,3212.858154,1376.909302,-0.407568,0.162937,0.128746,-0.407568
2311123,0.825493,0.444407,0.562467,1.093300,1.198564,0.829573,0.026412,3.165910,0.985186,0.639958,...,0.2871,0.148343,30.200001,165.946594,3793.733154,1356.562622,-0.471073,0.349970,0.104836,-0.264710
3431038,0.702893,0.594244,0.601256,1.099406,1.085055,0.717433,0.003272,2.856888,0.984503,0.719518,...,0.2291,0.203240,38.466999,15.304100,3206.133301,1416.060913,-0.404155,0.392491,0.123888,-0.286178


In [6]:
setting.pa+

NameError: name 'param' is not defined

In [15]:
X, Xv, yt, yv = train_test_split(
    data.drop(columns=[
        "rc_A"
    ])[train['type']=='1JHC'],
    y[train['type']=='1JHC'],
    test_size=0.3,
    random_state=233
)
lgbm = LGBMRegressor(objective='regression_l1', n_estimators=1000, learning_rate=0.1, subsample_freq=1, \
                     subsample=0.7, bagging_fraction=0.7, reg_alpha=0.1, reg_lambda=0.3, device_type='gpu', max_bin=63, **setting.param['1JHC'])
lgbm.fit(X, yt, eval_set=[(X, yt), (Xv, yv)], eval_metric='regression_l1', verbose=100)


[100]	training's l1: 0.832136	valid_1's l1: 0.96601
[200]	training's l1: 0.665744	valid_1's l1: 0.873444
[300]	training's l1: 0.585208	valid_1's l1: 0.84198
[400]	training's l1: 0.532823	valid_1's l1: 0.825467
[500]	training's l1: 0.496485	valid_1's l1: 0.815189
[600]	training's l1: 0.468199	valid_1's l1: 0.807372
[700]	training's l1: 0.446395	valid_1's l1: 0.802014
[800]	training's l1: 0.428097	valid_1's l1: 0.797592
[900]	training's l1: 0.413049	valid_1's l1: 0.794355
[1000]	training's l1: 0.400514	valid_1's l1: 0.791759


LGBMRegressor(bagging_fraction=0.7, boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, device_type='gpu', importance_type='split',
       learning_rate=0.1, max_bin=63, max_depth=19, min_child_samples=16,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=1000,
       n_jobs=-1, num_leaves=672, objective='regression_l1',
       random_state=None, reg_alpha=0.1, reg_lambda=0.3, silent=True,
       subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [ ]:
0.78855